In [ ]:
import pickle
import cv2
import mediapipe as mp
import numpy as np
from sklearn.tree import DecisionTreeClassifier

# Attempt to load the model
try:
    with open('C:/Users/LENOVO/Documents/mini project/model.p', 'rb') as file:
        model_dict = pickle.load(file)
    model = model_dict['model']
    print("Model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")
    model = None

# Initialize video capture
cap = cv2.VideoCapture(0)

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(static_image_mode=False, min_detection_confidence=0.3, max_num_hands=2)

# Define labels
labels_dict = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J', 10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'O', 15: 'P', 16: 'Q', 17: 'R', 18: 'S', 19: 'T', 20: 'U', 21: 'V', 22: 'W', 23: 'X', 24: 'Y', 25: 'Z'}

def normalize_features(features):
    # Normalize features if needed, e.g., between 0 and 1
    return features

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    if not ret:
        print("Failed to grab frame")
        break

    H, W, _ = frame.shape

    # Convert the frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with MediaPipe Hands
    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        combined_features = []

        for hand_landmarks in results.multi_hand_landmarks:
            x_ = [landmark.x for landmark in hand_landmarks.landmark]
            y_ = [landmark.y for landmark in hand_landmarks.landmark]

            # Extract features from each hand
            data_aux = []
            for i in range(21):
                data_aux.append(x_[i] - min(x_))
                data_aux.append(y_[i] - min(y_))

            combined_features.extend(data_aux)

        # Ensure combined_features length matches model expectation (84 features)
        if len(combined_features) != 84:
            print(f"Feature length mismatch: Expected 84, but got {len(combined_features)}")
            continue

        # Normalize features if needed
        combined_features = normalize_features(combined_features)

        # Predict the character
        if model is not None:
            try:
                prediction = model.predict([np.asarray(combined_features)])
                predicted_character = labels_dict.get(int(prediction[0]), "Unknown")
                print(f"Predicted Character: {predicted_character}")
            except Exception as e:
                print(f"Prediction error: {e}")
                predicted_character = "Error"
        else:
            predicted_character = "Model not available"

        # Draw landmarks and prediction on the frame
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())

        # Calculate bounding box for both hands combined
        all_x = [landmark.x for hand_landmarks in results.multi_hand_landmarks for landmark in hand_landmarks.landmark]
        all_y = [landmark.y for hand_landmarks in results.multi_hand_landmarks for landmark in hand_landmarks.landmark]

        x_min, x_max = min(all_x) * W, max(all_x) * W
        y_min, y_max = min(all_y) * H, max(all_y) * H

        x1 = int(x_min) - 10
        y1 = int(y_min) - 10
        x2 = int(x_max) + 10
        y2 = int(y_max) + 10

        # Draw bounding box and text
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 4)
        cv2.putText(frame, predicted_character, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 0), 3, cv2.LINE_AA)
    else:
        print("No hand landmarks detected")

    # Display the frame
    cv2.imshow('frame', frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close windows
cap.release()
cv2.destroyAllWindows()